In [235]:
import json
import copy
import pickle

import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [7]:
tg = json.load(open('ChatExport_2021-04-28/park_main.json'))['messages']
len(tg)

1745

In [8]:
df = pd.DataFrame(tg)
df.head()

,id,type,date,actor,actor_id,action,title,text,inviter,from,from_id,file,thumbnail,media_type,sticker_emoji,width,height,reply_to_message_id,members,message_id,edited,photo,forwarded_from,mime_type,duration_seconds,poll
0,1,service,2021-04-12T13:20:34,Парк Интуиции,channel1246350211,migrate_from_group,Парк Интуиции,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,service,2021-04-12T13:34:40,Dina,user149799201,join_group_by_link,NaN,,Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,service,2021-04-12T13:35:00,Anna Belostotskaya,user34022644,join_group_by_link,NaN,,Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,service,2021-04-12T13:38:51,Михаил Шубин,user383206754,join_group_by_link,NaN,,Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,service,2021-04-12T13:39:26,Андрей Мунтанион,user22982,join_group_by_link,NaN,,Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Описание полей

**id** - номер события  
**type** - тип события - 'service' или 'message'  
**date** - дата и время  
**title** - название чата  

Только для 'service':  
**actor** - имя  
**actor_id** - айди  
**action** - действие - 'migrate_from_group', 'join_group_by_link', 'invite_members',
       'pin_message', 'remove_members'  
**poll** - содержание опроса в JSON-формате  
**inviter** - кто пригласил  
**members** - имя того, кого пригласили или удалили из чата   
**message_id** - айди сообщения, с которым что-то сделали. Например, запинили  

Только для 'message':  
**text** - текст сообщения  
**from** - имя  
**from_id** - айди  
**reply_to_message_id** - айди сообщения, на которое ответили  
**edited** - дата и время, когда сообщение отредактировали  
**forwarded_from** - имя автора сообщения, которое форварднули  

Только для медиа:  
**file** - 'siticker', 'video_file', 'animation', nan (что-то другое, например - пдф)  
**thumbnail** - превью  
**media_type** - стикер или видос  
**sticker_emoji** - какой эмоджи соответствует стикеру  
**width** и **height** - размеры в пикселях видоса или стикера  
**photo** - файл фото  
**mime_type** - MIME тип медиа - 'video/mp4', 'text/html', 'application/pdf' и тд  
**duration_seconds** - продолжительность в секундах анимации или видоса  

In [40]:
df.loc[df.members.notnull()].tail()

,id,type,date,actor,actor_id,action,title,text,inviter,from,from_id,file,thumbnail,media_type,sticker_emoji,width,height,reply_to_message_id,members,message_id,edited,photo,forwarded_from,mime_type,duration_seconds,poll
37,38,service,2021-04-12T14:26:13,Eugene Arutyunov,user106338404,invite_members,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Vladimir 🔭 Tupikin],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1634,1640,service,2021-04-27T04:10:30,Полина Ладина,user707623959,remove_members,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Полина Ладина],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.mime_type.unique()

array([nan, 'video/mp4', 'text/html', 'application/pdf'], dtype=object)

In [51]:
df_messages = df.loc[df['type']=='message']
len(messages)

1707

In [42]:
users = messages['from_id'].drop_duplicates()
len(users)

41

#### Дропнем стикеры и всякое такое и оставим только нужные колонки

In [153]:
messages = df_messages.loc[df_messages.media_type.isnull()][['id', 'date', 'from', 'text', 'from_id', 'reply_to_message_id']]
len(messages)

1622

In [154]:
messages.head()

,id,date,from,text,from_id,reply_to_message_id
7,8,2021-04-12T13:49:56,Eugene Arutyunov,"господи, это будет самый грандиозный парк эва",user106338404,NaN
10,11,2021-04-12T13:50:54,Eugene Arutyunov,"наверное надо до завтра ещё потерпеть, чтобы в...",user106338404,NaN
13,14,2021-04-12T14:03:46,Tatsiana Kavalchuk,а до какого числа участников терпим?,user346294549,11.0
15,16,2021-04-12T14:05:46,Denis Domrachev,Всем привет),user64697041,NaN
16,17,2021-04-12T14:06:01,Eugene Arutyunov,не знаю ¯\_(ツ)_/¯\n\nну до завтра как минимум,user106338404,14.0


##### Посмотрим, сколько сообщений - ответы на другие сообщения

In [155]:
f'{len(messages.loc[messages.reply_to_message_id.notnull()])}/{len(messages)}'

'649/1622'

#### Заменим сообщения на их длину

In [156]:
messages.text = messages.text.apply(len)
messages = messages.rename({'text': 'text_length'}, axis=1)
messages.head()

,id,date,from,text_length,from_id,reply_to_message_id
7,8,2021-04-12T13:49:56,Eugene Arutyunov,45,user106338404,NaN
10,11,2021-04-12T13:50:54,Eugene Arutyunov,94,user106338404,NaN
13,14,2021-04-12T14:03:46,Tatsiana Kavalchuk,36,user346294549,11.0
15,16,2021-04-12T14:05:46,Denis Domrachev,12,user64697041,NaN
16,17,2021-04-12T14:06:01,Eugene Arutyunov,43,user106338404,14.0


#### Поправим типы колонок

In [157]:
messages.dtypes

id                       int64
date                    object
from                    object
text_length              int64
from_id                 object
reply_to_message_id    float64
dtype: object

In [158]:
messages.date = messages.astype({'date': 'datetime64'}).date
messages.reply_to_message_id.fillna(0, inplace=True)
messages.reply_to_message_id = messages.astype({'reply_to_message_id': 'int32'}).reply_to_message_id

In [159]:
messages.tail()

,id,date,from,text_length,from_id,reply_to_message_id
1738,1744,2021-04-27 09:28:20,Константин Константинопольский,3,user1261603,0
1739,1745,2021-04-27 09:28:32,Александр Филин,4,user96081342,1743
1740,1746,2021-04-27 09:28:41,Александр Филин,36,user96081342,0
1742,1748,2021-04-27 09:30:29,Oxana Tyulpinova,5,user60863562,0
1744,1750,2021-04-27 09:46:39,Natalia,19,user313688585,0


Что ещё хочется сделать:
- Сделать "Котел", куда отправляются сообщения без адресата
- Схлопывать сообщения идущие подряд в серии
- Определить, что значит "идущие подряд". С разницей меньше минуты?
- "Подрядность" нормировать на длину сообщения - длинные сообщения писать дольше
- Протестировать параметры на реальных ответах
- Ввести коэффициент затухания адресности - после, скажем, третьего сообщения подряд начинаем "подозревать", что сообщения отправляются в Котёл (то есть, человек думает вслух)
- Бенчмарк на угадывание адресата на тех случаях, когда адресат есть
- Считать адресными сообщения, отпревленные не через реплай, а с @меншном
- Адресатоми некоторых сообщения @могут @быть @несколько @человек

In [242]:
def flatten_replies(m_list):
    # Заменяет [{id: ...}, {id: ...}, ...] на {id1: {...}, id2: {...}}
    # Заменяет user_id автора на человеческое имя
    # Заменяет ссылку на сообщение на ссылку на автора. Если имя изменилось, она перезаписывается последним
    user_names = {msg['from_id']: msg['from'] for msg in m_list}
    user_names[0] = "Парк"
    
    m_dict = {}
    for msg in m_list:
        msg_c = copy.deepcopy(msg)

        if msg_c['reply_to_message_id'] != 0:
            msg_c['reply_to_user'] = user_names[m_dict.get(msg['reply_to_message_id'], {'from_id': 0})['from_id']]
        else:
            msg_c['reply_to_user'] = 'Парк'
        
        msg_c.pop('reply_to_message_id')
#         msg_c.pop('from_id')
        
        m_dict[msg['id']] = msg_c
        
    return {'users': user_names, 'messages': m_dict}

In [246]:
def to_d3_json(m_dict):
    d3_json = {'nodes': [], 'links': []}
    for uid in m_dict['users']:
        d3_json['nodes'].append({'id': m_dict['users'][uid]})

    messages = []
    for m in m_dict['messages']:
        from_user = m_dict['messages'][m]['from']
        to_user = m_dict['messages'][m]['reply_to_user']
        messages.append((from_user, to_user))
    pair_counts = Counter(messages)
    for pair, count in pair_counts.items():
        d3_json['links'].append({"source": pair[0], "target": pair[1], "value": count})

    with open('messages_counts.json', 'w') as outfile:
        json.dump(d3_json, outfile)
    return None

In [243]:
messages_dict = messages.to_dict(orient='records')

In [244]:
m = flatten_replies(messages_dict)

In [245]:
pickle.dump(m, open("messages_dict.pickle", "wb" ) )

In [ ]:
m['messages']